In [1]:
import numpy as np
import matplotlib.pyplot as py
from numpy import linalg as LA
from skimage import color, io
from skimage.metrics import peak_signal_noise_ratio as compare_psnr, structural_similarity as compare_ssim
import os

# Function to load an image and convert it to double
def load_image(file_path):
    image = io.imread(file_path)
    return image.astype('double')

# Function to calculate the mean of each channel
def calculate_mean(image):
    nb_pixels = image.shape[0] * image.shape[1]
    mean_colors = np.mean(image.reshape(-1, image.shape[2]), axis=0)
    return mean_colors

# Function to calculate the covariance matrix
def calculate_covariance(image, mean_colors):
    nb_pixels = image.shape[0] * image.shape[1]
    image_centered = image - mean_colors
    covariance_matrix = np.cov(image_centered.reshape(-1, image.shape[2]).T)
    return covariance_matrix

# Function to perform Karhunen-Loève Transform and return KLT basis and transformed image
def karhunen_loeve_transform(image, mean_colors):
    cov_matrix = calculate_covariance(image, mean_colors)
    eig_values, eig_vectors = LA.eig(cov_matrix)
    eig_vectors = eig_vectors.T
    transformed_image = np.dot(image - mean_colors, eig_vectors.T)
    return eig_vectors, transformed_image

# Function to reconstruct the image from the KLT basis and transformed image
def reconstruct_image(transformed_image, klt_basis, mean_colors):
    original_space_image = np.dot(transformed_image, klt_basis) + mean_colors
    return original_space_image

# Function to quantize the transformed image (placeholder, needs proper implementation based on the method from question 3)
def quantize_image(transformed_image):
    # Placeholder for quantization: For now, we will simply round the values to integers.
    # Replace with the appropriate quantization method described in question 3.
    quantized_image = np.round(transformed_image)
    return quantized_image

# Function to apply the entire process to a pair of images
def process_image_pair(image1_path, image2_path):
    # Load the images
    image1 = load_image(image1_path)
    image2 = load_image(image2_path)
    
    # Convert to a color space that provides the best quality/compression ratio
    # Assuming YCbCr based on typical usage, but this should be determined by question 3
    image1_ycbcr = color.rgb2ycbcr(image1)
    image2_ycbcr = color.rgb2ycbcr(image2)
    
    # Calculate the mean colors of image1
    mean_colors = calculate_mean(image1_ycbcr)
    
    # Perform KLT on image1
    klt_basis, image1_klt = karhunen_loeve_transform(image1_ycbcr, mean_colors)
    
    # Apply KLT on image2 using the basis from image1
    _, image2_klt = karhunen_loeve_transform(image2_ycbcr, mean_colors)
    
    # Quantize the transformed image2
    image2_quantized = quantize_image(image2_klt)
    
    # Reconstruct image2 from the quantized data
    image2_reconstructed = reconstruct_image(image2_quantized, klt_basis, mean_colors)
    image2_reconstructed = np.clip(image2_reconstructed, 0, 255).astype('uint8')
    
    # Convert back to RGB for quality metrics calculation
    image2_reconstructed_rgb = color.ycbcr2rgb(image2_reconstructed)
    image2_rgb = color.ycbcr2rgb(image2_ycbcr.astype('uint8'))
    
    # Calculate PSNR and SSIM
    psnr = compare_psnr(image2_rgb, image2_reconstructed_rgb)
    ssim = compare_ssim(image2_rgb, image2_reconstructed_rgb, multichannel=True)
    
    # Calculate compression rate (placeholder, should be based on actual compression data)
    # For actual compression rate, we would need to encode the quantized image and compare file sizes.
    compression_rate = 1.0 # Placeholder value
    
    return psnr, ssim, compression_rate, image2_reconstructed_rgb

image1_path = './images/kodim23.png'
image2_path = './images/kodim02.png'

psnr, ssim, compression_rate, image2_reconstructed = process_image_pair(image1_path, image2_path)


NameError: name 'np' is not defined